In [1]:
%matplotlib inline
import fastai
from fastai.vision.all import *
import cv2
from matplotlib import pyplot as plt
import re
import numpy as np
import chess
from PIL import Image

In [2]:
data_dir = Path("C:\Datasets\CHESS_BOARDS")
train_dir = data_dir / "train"
test_dir = data_dir / "test"

In [25]:
fnames = get_image_files(train_dir)
len(fnames)

80000

In [6]:
def fen_array(fen):
    ar = [[0 for y in range(8)] for x in range(8)]
    row, col = 0, 0
    for i, c in enumerate(fen):
        if c == "-" or c == "/":
            row += 1
            col = 0
        else:    
            if c.isnumeric():
                col += int(c)
            else:
                ar[row][col] = ord(c)
                col +=1
    return ar

def array_fen(ar):
    fen = ""
    for row, x in enumerate(ar):
        empty = 0
        for col, y in enumerate(x):
            if ar[row][col] == 0:
                empty += 1
                if col == 7:
                    fen += str(empty)
            else:
                if empty != 0:
                    fen += str(empty)
                empty = 0
                c = chr(ar[row][col])
                fen += c
                
            if col == 7 and row != 7:
                fen += "/"
    return fen

def true_fen(fen):
    return fen.replace("-", "/")

def get_label(l):
    if l == 0:
        return "Empty"
    else:
        return chr(l)

#divide the image into 8*8 parts (each cell on the board)
def get_images_cells(fnames):
    cells = []
    for i, f in enumerate(fnames):
        fen = true_fen(f.stem)

        print(f.name)

        img = cv2.imread(str(f))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        splitrows = np.split(img, 8, 0)

        blocks = []
        for rows in splitrows:    
            splitcells = np.split(rows, 8, 1)
            blocks.append(splitcells)

        cells.append((fen, blocks))
    return cells

In [28]:
cells = get_images_cells(fnames[1001:2000])

1b4r1-8-R7-8-r3K3-8-3pk3-8.jpeg
1B4R1-B7-8-8-1B1p4-4Kq1R-8-q1r2Nk1.jpeg
1B4R1-bP6-b6r-n1Q2P2-7k-8-q7-3b1K2.jpeg
1b4R1-K3kR2-1r6-r2n4-p7-3q2B1-R7-6R1.jpeg
1B4R1-K4B2-2r3R1-Rp6-8-6k1-1pN4P-4q3.jpeg
1B4r1-K7-8-R4rP1-1N3n2-8-pk1B2q1-3nB3.jpeg
1B4R1-K7-n1r5-7r-1RN5-1P1r2b1-kR6-bb6.jpeg
1b4R1-n5k1-K4n2-6b1-1q6-8-7N-8.jpeg
1B4R1-p4nn1-1R1K4-1P6-b7-4k2q-2nPQ3-8.jpeg
1b4R1-r1R3P1-8-1b4Pk-8-2p1q1p1-3K4-1N6.jpeg
1B4R1-R4Nn1-2r5-1Kb5-3rb3-7k-6b1-4r2r.jpeg
1B4r1-R7-B1Q5-3PK3-4P3-4k3-2b1P3-R3bn2.jpeg
1b4R1-rN2R3-8-qb1B4-B5K1-3Pb3-6k1-4rr2.jpeg
1b4RB-3R2PP-1Pq5-5r1b-1K1P4-8-8-5Q1k.jpeg
1B4RB-8-8-8-1K6-8-8-5Bk1.jpeg
1B4rk-6P1-8-1K6-8-4RP2-7p-1n6.jpeg
1b4Rn-4n3-4N3-5K2-Q7-5k2-4q3-7B.jpeg
1b4rN-8-4K3-5r2-R3b3-1b5r-1nb3k1-6N1.jpeg
1B4rQ-3r3p-2qKB3-8-4Q2k-2N3p1-5n2-3n4.jpeg
1B4RQ-p1P5-K2k2p1-4r3-N7-8-1P2Q3-1r5N.jpeg
1B5b-1K2p3-r5b1-6P1-8-4k3-2Q5-8.jpeg
1B5B-1K4N1-8-6B1-7k-1p1P4-5N2-3B4.jpeg
1B5b-1PK1k3-2R4B-8-8-2bn2B1-2N5-1BQ5.jpeg
1b5b-2B2P2-3P4-5P2-4b1k1-5p1P-P1n1K3-8.jpeg
1b5B-2P3n1-5k2-8-2K5-5P1Q-5p2-

1b6-1k6-5b2-K7-6p1-R6B-7B-2b3B1.jpeg
1b6-1K6-8-3k4-8-8-5R2-6b1.jpeg
1B6-1K6-8-3Q4-r3R2q-8-4k3-8.jpeg
1B6-1K6-8-4k2P-8-8-1b1r4-n7.jpeg
1b6-1K6-8-4n3-8-8-nP1k4-8.jpeg
1B6-1K6-8-6N1-1n3r2-8-6k1-1b6.jpeg
1b6-1K6-8-8-8-1N1p1kN1-8-8.jpeg
1b6-1K6-p7-8-Q3k2P-1R6-3B4-4nnq1.jpeg
1B6-1KB5-n4P2-8-8-8-2r5-1k5B.jpeg
1B6-1KN1k1n1-5p2-1Q6-4b3-8-4p3-1Nr1n1b1.jpeg
1B6-1Kn5-2p2r2-6b1-nk6-rnn3n1-7q-N7.jpeg
1B6-1n1b1Nn1-2p5-7p-8-r1R5-1K6-2rbk3.jpeg
1B6-1n1n4-1p6-2n4P-3p4-rK2k1nN-7b-2N5.jpeg
1b6-1n2N2P-Kp1Q4-N1r5-r7-7Q-1k4N1-4r1r1.jpeg
1b6-1N2N3-8-5q1R-3K1kn1-8-2P5-8.jpeg
1B6-1N4kP-1KQ1P3-6P1-b4B2-5bB1-5r2-7B.jpeg
1B6-1n4N1-7b-2K2k2-2p2q2-3Q4-8-n2NN3.jpeg
1B6-1N6-1K2r3-R7-r2N4-1p3b1r-5R2-7k.jpeg
1B6-1N6-1P6-2k5-8-8-K7-8.jpeg
1b6-1N6-8-8-8-3K1b2-6k1-8.jpeg
1B6-1N6-N4B2-3K4-3Q2P1-8-2R3k1-3b4.jpeg
1B6-1n6-pq1K4-4n3-bP6-P3n1B1-6B1-1N2k3.jpeg
1B6-1nK5-7r-8-8-2k5-3b4-3B4.jpeg
1B6-1npB3k-K2r4-2r5-1B6-2P5-1R2r2r-8.jpeg
1B6-1p1N2k1-2b2NP1-4P3-4NN2-K7-Q7-5Q2.jpeg
1b6-1P1pB3-4P3-1Kp3n1-6r1-2p5-1P2k3-4b3.jpeg
1b6-1p1R2

1B6-3r1R2-K5nP-1P3R1N-r7-3r4-1k6-6NN.jpeg
1b6-3R3q-6K1-P7-8-3p2k1-1N1b4-1nb3R1.jpeg
1B6-3R4-1b3K2-8-8-1q3k1p-8-8.jpeg
1B6-3r4-3b4-1R1n3N-1Qrr4-1K4k1-7P-2NR3r.jpeg
1b6-3r4-8-R7-7k-6BR-8-3K4.jpeg
1B6-3r4-b1nN4-8-1p1R1N1P-7k-2P5-5K2.jpeg
1B6-3RB3-8-7b-3K4-8-5k2-8.jpeg
1b6-3rPP2-K3B3-8-p1n4k-5nB1-r3b3-q1n5.jpeg
1B6-3rq1r1-2n5-5K2-B7-5k2-4bqp1-1r1r4.jpeg
1b6-4B1k1-PKRPq3-pr1p1b2-5P2-7B-8-8.jpeg
1b6-4B3-2rq4-4N3-Kn1nB2n-2r2k2-2r5-b1B5.jpeg
1b6-4b3-5k2-4Q3-8-8-8-2K5.jpeg
1B6-4b3-R1K1r3-6B1-q7-5nk1-8-8.jpeg
1B6-4Bn1q-5n2-7k-8-7r-8-4K1R1.jpeg
1B6-4bn2-2P5-3b1K2-5n2-4p3-5R2-7k.jpeg
1b6-4K2b-8-R3nR2-2R5-Qn6-1n2k3-6R1.jpeg
1b6-4K2k-r6r-4n3-8-8-6q1-1B6.jpeg
1B6-4k2P-7P-5q2-3P4-2Kp1R2-5p2-3b1N2.jpeg
1B6-4K3-1R6-6n1-8-8-P2pk3-8.jpeg
1B6-4k3-2K4b-Nq6-b3q3-1n1P1P1p-1b2q3-2r5.jpeg
1b6-4K3-3p1q2-5p2-r4Q2-6P1-q1pQ2k1-1r1B3B.jpeg
1B6-4k3-7q-p3p3-4r3-KN4n1-3rN3-B3N3.jpeg
1B6-4k3-8-1r1R1R2-3K4-5q2-6p1-7N.jpeg
1b6-4k3-8-2K2p2-3P4-2Np1B2-n4r2-8.jpeg
1b6-4K3-8-4q3-4r3-5bk1-2p1P3-7n.jpeg
1B6-4K3-8-6N1-8-2p5-1N5p

1b6-8-2b5-1K6-8-b2pN3-2n5-5kR1.jpeg
1B6-8-2b5-5Pk1-8-8-1R5N-4K3.jpeg
1b6-8-2K1P3-5Q1k-3R3P-5P2-8-7n.jpeg
1b6-8-2K5-1p6-3B2kN-2Br1B2-N1N5-8.jpeg
1B6-8-2K5-2P5-4PP1r-2N2N2-2Q5-4kb2.jpeg
1b6-8-2K5-6r1-pp1N1p1Q-4k3-1r1p1B2-1q4bQ.jpeg
1B6-8-2K5-8-7r-bqk5-7Q-8.jpeg
1b6-8-2Kb4-p1p5-8-3n4-Pq6-7k.jpeg
1B6-8-2n5-8-5K1r-8-7k-8.jpeg
1b6-8-2P1Rb2-2K3Q1-5nQ1-B1R5-5n2-4k1N1.jpeg
1B6-8-2PR4-4K3-r7-2R1r3-3n4-3B2k1.jpeg
1B6-8-2q5-1bBQ4-K7-1n4q1-3b1bR1-B3k1N1.jpeg
1B6-8-2R1BbN1-1K1n2B1-Br1P4-4N3-2n5-1k4b1.jpeg
1B6-8-2r4P-7R-1Nk4p-P2n4-6rN-1K6.jpeg
1B6-8-2R5-8-3k4-K5Q1-8-8.jpeg
1B6-8-3bn3-B5k1-3N3N-8-K3R3-8.jpeg
1b6-8-3k1B2-4p3-8-7p-8-K5n1.jpeg
1B6-8-3K2B1-8-8-6R1-8-5k2.jpeg
1b6-8-3K4-3n4-8-8-3n3n-7k.jpeg
1B6-8-3K4-3P1R2-1P1q1p2-8-7P-2q1k3.jpeg
1b6-8-3K4-8-1R6-8-6k1-1b6.jpeg
1B6-8-3K4-8-8-8-4k1B1-6q1.jpeg
1b6-8-3N1p1N-3k2p1-K2p4-1R3B2-8-1R3n2.jpeg
1B6-8-3N4-1n2k3-8-1q6-2KP3P-7N.jpeg
1B6-8-3N4-8-7r-8-1K4k1-rQ6.jpeg
1b6-8-3N4-8-P5n1-KP1p1n2-8-knQ5.jpeg
1b6-8-3Nr3-bk4P1-1b1N1RN1-8-K4B2-7N.jpeg
1b6-8-3p1k2-8-

1b6-R4Bk1-2p5-4Kp2-1p1P3b-8-1p5N-1nN1B3.jpeg
1B6-r4K2-7Q-7k-N7-1R2r3-7R-5n2.jpeg
1B6-r4R2-K2Q4-1P6-1R1k4-b3Qr2-1b5Q-8.jpeg
1b6-r5NP-8-4k3-2K5-6qP-8-6r1.jpeg
1B6-r6q-8-6k1-3n3n-6Q1-8-4K3.jpeg
1b6-R7-1K1r3p-2b2rb1-3br3-4R3-6Pk-5b2.jpeg
1B6-R7-5Nk1-1BP5-8-8-5K2-4n3.jpeg
1b6-R7-6P1-2P5-2K2k2-8-8-1N5b.jpeg
1B6-RK4P1-1bB2kB1-P4RQ1-1B6-8-Q7-8.jpeg
1b6-RN1Q4-P7-K6k-2n5-7N-7p-6R1.jpeg
1B6-RN6-8-2nN4-8-1K3R2-6Rk-8.jpeg
1B6-rq6-8-5K2-7B-8-6k1-4R3.jpeg
1bb1B3-4Bnp1-1K4k1-3r4-8-1Q6-8-R2R4.jpeg
1Bb1b3-p4P2-2N5-8-K7-8-k5rR-1N6.jpeg
1BB1bN2-5N2-2p5-2r1K3-8-1P6-Q4n1P-R4N1k.jpeg
1BB1bR2-4r3-4K3-8-p4R1k-8-3r4-5R2.jpeg
1Bb1k3-8-K5n1-8-8-2Q5-1R6-8.jpeg
1BB1K3-R3B3-8-8-Q3nPr1-2qQ4-2R5-2q1R1k1.jpeg
1bB1kb2-2K3N1-7B-1B6-5n2-1R1R2R1-3N1b2-2r5.jpeg
1bb1KQ2-3Q1B1P-8-8-1p1p2N1-4k3-N2B4-8.jpeg
1bB1Kq2-4n3-1Q3p1k-1N3BN1-8-8-1BR2n2-3Q4.jpeg
1bb1n1r1-8-2B5-8-2P2B2-3k2N1-r3rR1P-K5N1.jpeg
1bB1N3-3K2Q1-8-N1P5-4p3-7p-2Q2B2-7k.jpeg
1Bb1N3-3n3p-7p-p1R4p-n2q4-K6k-4r3-8.jpeg
1BB1nr2-2pPp3-K4N2-4P3-7R-4q2k-8-2Q5.jpeg
1bB1R3-1

# Generating new data for pieces classification

In [11]:
data_gen = Path("C:\Datasets\CHESS_PIECES")
data_gen_train = data_gen / "train"
data_gen_test = data_gen / "test"

In [29]:
for i, cell in enumerate(cells):
    fen = cell[0]
    arfen = fen_array(fen)
    
    blocks = cell[1]
    for y, row in enumerate(blocks):
        
        #limit to x label 0 (empty)          
        empty_count = 0
        
        for x, block in enumerate(row):
            label = arfen[y][x] or 0
            
            if label == 0:
                empty_count += 1
            
            # x empty per row limit
            if label != 0 or empty_count < 2:
                
                folder = data_gen_train / str(label)

                if str(label) not in os.listdir(data_gen_train):
                    os.makedirs(folder)

                files_count = len(os.listdir(folder))
                fname = files_count + 1
                fname = str(fname) + ".jpeg"

                im = Image.fromarray(block)
                im.save(folder / fname)

                print(folder / fname)

C:\Datasets\CHESS_PIECES\train\0\7985.jpeg
C:\Datasets\CHESS_PIECES\train\98\1458.jpeg
C:\Datasets\CHESS_PIECES\train\114\938.jpeg
C:\Datasets\CHESS_PIECES\train\0\7986.jpeg
C:\Datasets\CHESS_PIECES\train\82\893.jpeg
C:\Datasets\CHESS_PIECES\train\0\7987.jpeg
C:\Datasets\CHESS_PIECES\train\0\7988.jpeg
C:\Datasets\CHESS_PIECES\train\114\939.jpeg
C:\Datasets\CHESS_PIECES\train\0\7989.jpeg
C:\Datasets\CHESS_PIECES\train\75\999.jpeg
C:\Datasets\CHESS_PIECES\train\0\7990.jpeg
C:\Datasets\CHESS_PIECES\train\0\7991.jpeg
C:\Datasets\CHESS_PIECES\train\112\956.jpeg
C:\Datasets\CHESS_PIECES\train\107\999.jpeg
C:\Datasets\CHESS_PIECES\train\0\7992.jpeg
C:\Datasets\CHESS_PIECES\train\0\7993.jpeg
C:\Datasets\CHESS_PIECES\train\66\1440.jpeg
C:\Datasets\CHESS_PIECES\train\82\894.jpeg
C:\Datasets\CHESS_PIECES\train\66\1441.jpeg
C:\Datasets\CHESS_PIECES\train\0\7994.jpeg
C:\Datasets\CHESS_PIECES\train\0\7995.jpeg
C:\Datasets\CHESS_PIECES\train\0\7996.jpeg
C:\Datasets\CHESS_PIECES\train\0\7997.jpeg
C:\D

C:\Datasets\CHESS_PIECES\train\0\8061.jpeg
C:\Datasets\CHESS_PIECES\train\0\8062.jpeg
C:\Datasets\CHESS_PIECES\train\112\963.jpeg
C:\Datasets\CHESS_PIECES\train\113\466.jpeg
C:\Datasets\CHESS_PIECES\train\112\964.jpeg
C:\Datasets\CHESS_PIECES\train\0\8063.jpeg
C:\Datasets\CHESS_PIECES\train\75\1008.jpeg
C:\Datasets\CHESS_PIECES\train\0\8064.jpeg
C:\Datasets\CHESS_PIECES\train\78\938.jpeg
C:\Datasets\CHESS_PIECES\train\0\8065.jpeg
C:\Datasets\CHESS_PIECES\train\66\1452.jpeg
C:\Datasets\CHESS_PIECES\train\82\913.jpeg
C:\Datasets\CHESS_PIECES\train\82\914.jpeg
C:\Datasets\CHESS_PIECES\train\0\8066.jpeg
C:\Datasets\CHESS_PIECES\train\78\939.jpeg
C:\Datasets\CHESS_PIECES\train\110\981.jpeg
C:\Datasets\CHESS_PIECES\train\0\8067.jpeg
C:\Datasets\CHESS_PIECES\train\114\951.jpeg
C:\Datasets\CHESS_PIECES\train\0\8068.jpeg
C:\Datasets\CHESS_PIECES\train\75\1009.jpeg
C:\Datasets\CHESS_PIECES\train\98\1471.jpeg
C:\Datasets\CHESS_PIECES\train\0\8069.jpeg
C:\Datasets\CHESS_PIECES\train\114\952.jpeg
C

C:\Datasets\CHESS_PIECES\train\78\946.jpeg
C:\Datasets\CHESS_PIECES\train\0\8145.jpeg
C:\Datasets\CHESS_PIECES\train\66\1466.jpeg
C:\Datasets\CHESS_PIECES\train\98\1486.jpeg
C:\Datasets\CHESS_PIECES\train\0\8146.jpeg
C:\Datasets\CHESS_PIECES\train\75\1019.jpeg
C:\Datasets\CHESS_PIECES\train\112\970.jpeg
C:\Datasets\CHESS_PIECES\train\114\968.jpeg
C:\Datasets\CHESS_PIECES\train\0\8147.jpeg
C:\Datasets\CHESS_PIECES\train\98\1487.jpeg
C:\Datasets\CHESS_PIECES\train\0\8148.jpeg
C:\Datasets\CHESS_PIECES\train\80\949.jpeg
C:\Datasets\CHESS_PIECES\train\0\8149.jpeg
C:\Datasets\CHESS_PIECES\train\0\8150.jpeg
C:\Datasets\CHESS_PIECES\train\107\1019.jpeg
C:\Datasets\CHESS_PIECES\train\0\8151.jpeg
C:\Datasets\CHESS_PIECES\train\81\506.jpeg
C:\Datasets\CHESS_PIECES\train\0\8152.jpeg
C:\Datasets\CHESS_PIECES\train\0\8153.jpeg
C:\Datasets\CHESS_PIECES\train\66\1467.jpeg
C:\Datasets\CHESS_PIECES\train\66\1468.jpeg
C:\Datasets\CHESS_PIECES\train\0\8154.jpeg
C:\Datasets\CHESS_PIECES\train\75\1020.jpeg


C:\Datasets\CHESS_PIECES\train\0\8233.jpeg
C:\Datasets\CHESS_PIECES\train\98\1504.jpeg
C:\Datasets\CHESS_PIECES\train\66\1488.jpeg
C:\Datasets\CHESS_PIECES\train\0\8234.jpeg
C:\Datasets\CHESS_PIECES\train\81\510.jpeg
C:\Datasets\CHESS_PIECES\train\114\971.jpeg
C:\Datasets\CHESS_PIECES\train\0\8235.jpeg
C:\Datasets\CHESS_PIECES\train\78\953.jpeg
C:\Datasets\CHESS_PIECES\train\114\972.jpeg
C:\Datasets\CHESS_PIECES\train\0\8236.jpeg
C:\Datasets\CHESS_PIECES\train\107\1030.jpeg
C:\Datasets\CHESS_PIECES\train\0\8237.jpeg
C:\Datasets\CHESS_PIECES\train\0\8238.jpeg
C:\Datasets\CHESS_PIECES\train\66\1489.jpeg
C:\Datasets\CHESS_PIECES\train\75\1030.jpeg
C:\Datasets\CHESS_PIECES\train\98\1505.jpeg
C:\Datasets\CHESS_PIECES\train\0\8239.jpeg
C:\Datasets\CHESS_PIECES\train\82\931.jpeg
C:\Datasets\CHESS_PIECES\train\78\954.jpeg
C:\Datasets\CHESS_PIECES\train\0\8240.jpeg
C:\Datasets\CHESS_PIECES\train\114\973.jpeg
C:\Datasets\CHESS_PIECES\train\0\8241.jpeg
C:\Datasets\CHESS_PIECES\train\66\1490.jpeg


C:\Datasets\CHESS_PIECES\train\0\8318.jpeg
C:\Datasets\CHESS_PIECES\train\0\8319.jpeg
C:\Datasets\CHESS_PIECES\train\75\1040.jpeg
C:\Datasets\CHESS_PIECES\train\0\8320.jpeg
C:\Datasets\CHESS_PIECES\train\78\960.jpeg
C:\Datasets\CHESS_PIECES\train\114\980.jpeg
C:\Datasets\CHESS_PIECES\train\0\8321.jpeg
C:\Datasets\CHESS_PIECES\train\98\1528.jpeg
C:\Datasets\CHESS_PIECES\train\98\1529.jpeg
C:\Datasets\CHESS_PIECES\train\0\8322.jpeg
C:\Datasets\CHESS_PIECES\train\0\8323.jpeg
C:\Datasets\CHESS_PIECES\train\114\981.jpeg
C:\Datasets\CHESS_PIECES\train\82\940.jpeg
C:\Datasets\CHESS_PIECES\train\0\8324.jpeg
C:\Datasets\CHESS_PIECES\train\0\8325.jpeg
C:\Datasets\CHESS_PIECES\train\98\1530.jpeg
C:\Datasets\CHESS_PIECES\train\110\1003.jpeg
C:\Datasets\CHESS_PIECES\train\0\8326.jpeg
C:\Datasets\CHESS_PIECES\train\75\1041.jpeg
C:\Datasets\CHESS_PIECES\train\78\961.jpeg
C:\Datasets\CHESS_PIECES\train\0\8327.jpeg
C:\Datasets\CHESS_PIECES\train\107\1041.jpeg
C:\Datasets\CHESS_PIECES\train\0\8328.jpeg


...

C:\Datasets\CHESS_PIECES\train\0\15661.jpeg
C:\Datasets\CHESS_PIECES\train\75\1958.jpeg
C:\Datasets\CHESS_PIECES\train\107\1958.jpeg
C:\Datasets\CHESS_PIECES\train\0\15662.jpeg
C:\Datasets\CHESS_PIECES\train\0\15663.jpeg
C:\Datasets\CHESS_PIECES\train\0\15664.jpeg
C:\Datasets\CHESS_PIECES\train\78\1682.jpeg
C:\Datasets\CHESS_PIECES\train\98\2670.jpeg
C:\Datasets\CHESS_PIECES\train\0\15665.jpeg
C:\Datasets\CHESS_PIECES\train\66\2645.jpeg
C:\Datasets\CHESS_PIECES\train\82\1637.jpeg
C:\Datasets\CHESS_PIECES\train\75\1959.jpeg
C:\Datasets\CHESS_PIECES\train\0\15666.jpeg
C:\Datasets\CHESS_PIECES\train\80\1811.jpeg
C:\Datasets\CHESS_PIECES\train\0\15667.jpeg
C:\Datasets\CHESS_PIECES\train\98\2671.jpeg
C:\Datasets\CHESS_PIECES\train\66\2646.jpeg
C:\Datasets\CHESS_PIECES\train\107\1959.jpeg
C:\Datasets\CHESS_PIECES\train\66\2647.jpeg
C:\Datasets\CHESS_PIECES\train\80\1812.jpeg
C:\Datasets\CHESS_PIECES\train\0\15668.jpeg
C:\Datasets\CHESS_PIECES\train\82\1638.jpeg
C:\Datasets\CHESS_PIECES\train

C:\Datasets\CHESS_PIECES\train\0\15744.jpeg
C:\Datasets\CHESS_PIECES\train\113\852.jpeg
C:\Datasets\CHESS_PIECES\train\82\1655.jpeg
C:\Datasets\CHESS_PIECES\train\107\1968.jpeg
C:\Datasets\CHESS_PIECES\train\0\15745.jpeg
C:\Datasets\CHESS_PIECES\train\98\2680.jpeg
C:\Datasets\CHESS_PIECES\train\66\2663.jpeg
C:\Datasets\CHESS_PIECES\train\107\1969.jpeg
C:\Datasets\CHESS_PIECES\train\98\2681.jpeg
C:\Datasets\CHESS_PIECES\train\0\15746.jpeg
C:\Datasets\CHESS_PIECES\train\75\1969.jpeg
C:\Datasets\CHESS_PIECES\train\78\1692.jpeg
C:\Datasets\CHESS_PIECES\train\0\15747.jpeg
C:\Datasets\CHESS_PIECES\train\66\2664.jpeg
C:\Datasets\CHESS_PIECES\train\0\15748.jpeg
C:\Datasets\CHESS_PIECES\train\66\2665.jpeg
C:\Datasets\CHESS_PIECES\train\0\15749.jpeg
C:\Datasets\CHESS_PIECES\train\110\1695.jpeg
C:\Datasets\CHESS_PIECES\train\0\15750.jpeg
C:\Datasets\CHESS_PIECES\train\82\1656.jpeg
C:\Datasets\CHESS_PIECES\train\82\1657.jpeg
C:\Datasets\CHESS_PIECES\train\82\1658.jpeg
C:\Datasets\CHESS_PIECES\trai

C:\Datasets\CHESS_PIECES\train\0\15813.jpeg
C:\Datasets\CHESS_PIECES\train\0\15814.jpeg
C:\Datasets\CHESS_PIECES\train\110\1702.jpeg
C:\Datasets\CHESS_PIECES\train\0\15815.jpeg
C:\Datasets\CHESS_PIECES\train\0\15816.jpeg
C:\Datasets\CHESS_PIECES\train\82\1666.jpeg
C:\Datasets\CHESS_PIECES\train\0\15817.jpeg
C:\Datasets\CHESS_PIECES\train\98\2692.jpeg
C:\Datasets\CHESS_PIECES\train\66\2682.jpeg
C:\Datasets\CHESS_PIECES\train\66\2683.jpeg
C:\Datasets\CHESS_PIECES\train\110\1703.jpeg
C:\Datasets\CHESS_PIECES\train\110\1704.jpeg
C:\Datasets\CHESS_PIECES\train\0\15818.jpeg
C:\Datasets\CHESS_PIECES\train\75\1978.jpeg
C:\Datasets\CHESS_PIECES\train\0\15819.jpeg
C:\Datasets\CHESS_PIECES\train\112\1838.jpeg
C:\Datasets\CHESS_PIECES\train\0\15820.jpeg
C:\Datasets\CHESS_PIECES\train\0\15821.jpeg
C:\Datasets\CHESS_PIECES\train\110\1705.jpeg
C:\Datasets\CHESS_PIECES\train\80\1824.jpeg
C:\Datasets\CHESS_PIECES\train\107\1978.jpeg
C:\Datasets\CHESS_PIECES\train\0\15822.jpeg
C:\Datasets\CHESS_PIECES\t

C:\Datasets\CHESS_PIECES\train\0\15889.jpeg
C:\Datasets\CHESS_PIECES\train\98\2708.jpeg
C:\Datasets\CHESS_PIECES\train\66\2699.jpeg
C:\Datasets\CHESS_PIECES\train\98\2709.jpeg
C:\Datasets\CHESS_PIECES\train\0\15890.jpeg
C:\Datasets\CHESS_PIECES\train\66\2700.jpeg
C:\Datasets\CHESS_PIECES\train\0\15891.jpeg
C:\Datasets\CHESS_PIECES\train\0\15892.jpeg
C:\Datasets\CHESS_PIECES\train\78\1715.jpeg
C:\Datasets\CHESS_PIECES\train\0\15893.jpeg
C:\Datasets\CHESS_PIECES\train\107\1987.jpeg
C:\Datasets\CHESS_PIECES\train\0\15894.jpeg
C:\Datasets\CHESS_PIECES\train\98\2710.jpeg
C:\Datasets\CHESS_PIECES\train\80\1835.jpeg
C:\Datasets\CHESS_PIECES\train\75\1987.jpeg
C:\Datasets\CHESS_PIECES\train\0\15895.jpeg
C:\Datasets\CHESS_PIECES\train\78\1716.jpeg
C:\Datasets\CHESS_PIECES\train\0\15896.jpeg
C:\Datasets\CHESS_PIECES\train\110\1714.jpeg
C:\Datasets\CHESS_PIECES\train\98\2711.jpeg
C:\Datasets\CHESS_PIECES\train\0\15897.jpeg
C:\Datasets\CHESS_PIECES\train\66\2701.jpeg
C:\Datasets\CHESS_PIECES\train

C:\Datasets\CHESS_PIECES\train\0\15965.jpeg
C:\Datasets\CHESS_PIECES\train\114\1707.jpeg
C:\Datasets\CHESS_PIECES\train\0\15966.jpeg
C:\Datasets\CHESS_PIECES\train\98\2727.jpeg
C:\Datasets\CHESS_PIECES\train\107\1996.jpeg
C:\Datasets\CHESS_PIECES\train\0\15967.jpeg
C:\Datasets\CHESS_PIECES\train\112\1849.jpeg
C:\Datasets\CHESS_PIECES\train\0\15968.jpeg
C:\Datasets\CHESS_PIECES\train\82\1679.jpeg
C:\Datasets\CHESS_PIECES\train\0\15969.jpeg
C:\Datasets\CHESS_PIECES\train\66\2720.jpeg
C:\Datasets\CHESS_PIECES\train\98\2728.jpeg
C:\Datasets\CHESS_PIECES\train\82\1680.jpeg
C:\Datasets\CHESS_PIECES\train\110\1722.jpeg
C:\Datasets\CHESS_PIECES\train\0\15970.jpeg
C:\Datasets\CHESS_PIECES\train\98\2729.jpeg
C:\Datasets\CHESS_PIECES\train\112\1850.jpeg
C:\Datasets\CHESS_PIECES\train\0\15971.jpeg
C:\Datasets\CHESS_PIECES\train\80\1846.jpeg
C:\Datasets\CHESS_PIECES\train\78\1723.jpeg
C:\Datasets\CHESS_PIECES\train\0\15972.jpeg
C:\Datasets\CHESS_PIECES\train\78\1724.jpeg
C:\Datasets\CHESS_PIECES\tr